<a href="https://colab.research.google.com/github/midhun-james/val-mod-with-gliner/blob/main/gliner.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install gliner rapidfuzz

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.0/63.0 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 37.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 92.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 81.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 43.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 91.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4

In [2]:
pip install polars

In [4]:
!pip install fastexcel

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 13.6 MB/s eta 0:00:00


In [5]:
import pandas as pd
import json
from collections import defaultdict
import os
import time
import gzip
import sqlite3
import sqlparse
from sqlparse.sql import Token
from sqlparse.tokens import Literal,String
from datetime import datetime
import random
import re
import string
import polars as pl
from openpyxl import load_workbook
from gliner import GLiNER
from rapidfuzz import process, fuzz

class DataMaskerCSV:
    def __init__(self,file_path):
        # self.entity_column_map={
        #                 'names': 'names',
        #                 'emails': 'emails',
        #                 'phone': 'phone',
        #                 'credit': 'credit',
        #                 'url': 'url',
        #                 'location': 'location',
        #                 'company': 'company',
        #             }
        self.file_path=file_path
        self.base_name=os.path.splitext(os.path.basename(self.file_path))[0]
        self.output_dir=self.base_name
        os.makedirs(self.output_dir, exist_ok=True)
        self.entity_column_map={
                'name': 'company',
                'domain': 'url',
                'locality': 'location',
                }
        self.model=GLiNER.from_pretrained("urchade/gliner_base")
        self.sensitive_columns = self.entity_column_map.keys()
        start=time.time()
        self.faker_data_path= 'faker_dataset_v3.json.gz'
        with gzip.open(self.faker_data_path, 'rt',encoding='utf-8') as f:
            faker_list = json.load(f)
        end=time.time()
        print(f"⏳ Faker data loaded in {end-start:.6f} seconds")
        self.faker_data = {}
        for d in faker_list:
            self.faker_data.update(d)
        self.domain_pool= self.faker_data['url']
        self.forward_mapping = defaultdict(dict)
        self.backward_mapping = defaultdict(dict)
        self.mapping= defaultdict(dict)
        self.fake_data_index = defaultdict(int)
        self.used_fakes = defaultdict(set)
        self.used_urls = set()
        self.url_extensions =  [
                                    ".com", ".net", ".org", ".edu", ".gov", ".co", ".us", ".uk", ".in", ".ru",
                                    ".jp", ".cn", ".de", ".fr", ".it", ".nl", ".es", ".br", ".au", ".ca",
                                    ".ch", ".se", ".no", ".za", ".mx", ".ar", ".be", ".kr", ".pl", ".tr",
                                    ".ua", ".ir", ".sa", ".ae", ".my", ".sg", ".hk", ".tw", ".nz", ".id",
                                    ".th", ".ph", ".vn", ".bd", ".lk", ".np", ".pk", ".cz", ".gr", ".hu",
                                    ".fi", ".dk", ".il", ".ie", ".pt", ".sk", ".si", ".ro", ".bg", ".rs",
                                    ".lt", ".lv", ".ee", ".hr", ".ba", ".md", ".ge", ".kz", ".by", ".tm",
                                    ".uz", ".af", ".qa", ".om", ".kw", ".bh", ".ye", ".jo", ".lb", ".sy",
                                    ".iq", ".ps", ".az", ".am", ".kg", ".mn", ".bt", ".mv", ".mm", ".kh",
                                    ".la", ".tl", ".sb", ".fj", ".pg", ".to", ".tv", ".ws", ".fm", ".ki"
                                ]


    @staticmethod
    def time_it(func):
        def wrapper(*args, **kwargs):
            start = time.time()
            result = func(*args, **kwargs)
            end = time.time()
            print(f'\n⏳ Execution time {func.__name__}: {end-start:.6f} seconds')
            return result
        return wrapper

    @time_it
    def csv_extraction(self):

        output_csv_path=os.path.join(self.output_dir,f'new_{self.base_name}.csv')
        if self.file_path.endswith('.xlsx'):
            sheet_names = pd.ExcelFile(self.file_path).sheet_names
            df=pl.read_excel(self.file_path,engine='calamine',sheet_name=sheet_names)
            combined_df = pl.concat(df.values(), how="diagonal")
            combined_df.write_csv('intermediate.csv')
            self.file_path='intermediate.csv'

        all_data={}
        for col in self.sensitive_columns:
            entity=self.entity_column_map.get(col)
            if entity:
                all_data.setdefault(entity, [])
        df=pd.read_csv(self.file_path)
        for col in self.sensitive_columns:
            if col in df.columns:
                entity=self.entity_column_map.get(col)
                if entity:
                    values=df[col].dropna().to_list()
                    all_data[entity].extend(values)
                else:
                    entity=self.entity_column_map.get(col.lower())
                    if entity:
                        all_data[entity].extend([None]*len(df))
        max_len=max([len(v) for v in all_data.values()])
        for entity in all_data:
            all_data[entity].extend([None]*(max_len-len(all_data[entity])))
        final_df=pd.DataFrame(all_data)
        final_df.to_csv(output_csv_path,index=False)

        if self.file_path == 'intermediate.csv': os.remove(self.file_path)
        self.anonymize_csv(output_csv_path)

    def _get_fake_value(self, entity, original_value):
        """Return consistent fake value for an original value."""
        col_key =  entity  # default fallback if column not passed


        if original_value in self.forward_mapping[col_key]:
            return self.forward_mapping[col_key][original_value]
        if entity =='url':
            while True:
                domain1,domain2=random.sample(self.domain_pool,2)
                fake_value=f"https://{domain1.lower()}.{domain2.lower()}.co"
                if fake_value not in self.used_fakes[entity]:
                    break
            self.used_fakes[entity].add(fake_value)
            self.forward_mapping[col_key][original_value] = fake_value
            self.backward_mapping[col_key][fake_value] = original_value
            return fake_value

        while self.fake_data_index[entity] < len(self.faker_data[entity]):
            fake_value = self.faker_data[entity][self.fake_data_index[entity]]
            self.fake_data_index[entity] += 1

            if fake_value not in self.used_fakes[entity]:
                self.used_fakes[entity].add(fake_value)
                self.forward_mapping[col_key][original_value] = fake_value
                self.backward_mapping[col_key][fake_value] = original_value
                return fake_value

        counter=1
        base_fake_value=original_value
        while True:
            fallback_value= self.modify_fake_value(entity, base_fake_value,  counter=counter)
            if fallback_value not in self.used_fakes[entity]:
                self.used_fakes[entity].add(fallback_value)
                self.forward_mapping[col_key][original_value] = fallback_value
                self.backward_mapping[col_key][fallback_value] = original_value
                return fallback_value
            counter+=1


    def modify_fake_value(self,entity,original_value,counter=1):
        """Modify the fake value to ensure uniqueness."""
        if entity=="names":
            base=random.choice(self.faker_data['names'])
            return base+f"{string.ascii_lowercase[counter % 26]}"
        elif entity=="emails":
            base=random.choice(self.faker_data['emails'])
            name,domain=base.split('@')
            return f"{name}{counter}@{domain}"
        elif entity=="url":
            fake_value=original_value
            while fake_value in self.used_urls:
                ext=random.choice(self.url_extensions)
                if not fake_value.endswith(ext):
                    fake_value=fake_value+ext
            self.used_urls.add(fake_value)
            return fake_value
        elif entity=="phone":
            base=random.choice(self.faker_data['phone'])
            return f"{base[:-2]}{counter % 100:02d}"
        elif entity == "company":
            base=random.choice(self.faker_data['company'])
            return f"{base} Group {counter % 100_000_000 + 1}"
        elif entity == "credit":
            return f"{original_value[:-4]}{counter % 10000:04d}"
        else:
            return f"{original_value}-{counter}"

    @time_it
    def anonymize_csv(self, input_csv_path):
        df = pd.read_csv(input_csv_path)
        for entity in df.columns:

            if entity not in self.faker_data:
                print(f"Warning: No fake data available for entity type '{entity}' '.")
                continue

            df[entity] = df[entity].apply(lambda val: self._get_fake_value(entity, val) if pd.notna(val) else val)

        output_csv_path=os.path.join(self.output_dir,f'{self.base_name}_masked.csv')
        df.to_csv(output_csv_path, index=False)

        combined_mapping = {
            "metadata": {
                "timestamp": datetime.now().isoformat(),
                "columns_anonymized": list(self.forward_mapping.keys()),
                "total_entries": {
                    col: len(self.forward_mapping[col]) for col in self.forward_mapping
                }
            },
            "forward_mapping": self.forward_mapping,
            "backward_mapping": self.backward_mapping,
        }
        map_path =f'{self.base_name}_mapping.json'
        with open(map_path, 'w') as f:
            json.dump(combined_mapping, f, indent=2)


        # print(f"Anonymized CSV saved to: {output_csv_path}")
        print(f" mapping saved to: {map_path}")


    @time_it
    def deanonymize_csv(self,anonymized_csv_path,map_path,deanonymized_csv_path):
        df = pd.read_csv(anonymized_csv_path)

        with open(map_path, 'r') as f:
            self.backward_mapping = json.load(f).get("backward_mapping", {})

        for col in self.sensitive_columns:
            entity= self.entity_column_map.get(col.lower())
            if col not in df.columns:
                continue
            backward_map = self.backward_mapping.get(entity, {})

            df[col]=df[col].apply(lambda val:backward_map.get(val,entity) if pd.notna(val) else val )
        df.to_csv(deanonymized_csv_path,index=False)
        print(f"Deanonymized CSV saved to: {deanonymized_csv_path}")
    @time_it
    def csv_to_sql(self,csv_path,_db_path,table_name):
        try:
            df=pd.read_csv(csv_path)
            conn=sqlite3.connect(_db_path)
            df.to_sql(table_name,conn,if_exists='replace',index=False)
            conn.close()
        except Exception as e:
            print(f"❌ Failed to import CSV: {e}")

    def correct_word(self,word,threshold=65):
      valid_list=[company.lower() for company in self.forward_mapping['company'].keys()]
      match=process.extractOne(word,valid_list,scorer=fuzz.ratio)
      return match[0] if match and match[1]>=threshold else word

    def find_and_correct_entities(self,text):
      valid_company=[company.lower() for company in self.forward_mapping['company'].keys()]
      entities=self.model.predict_entities(text,labels=["person","organization"])

      corrected_entities=[]
      for ent in entities:
        entity_text=ent['text']
        start=ent['start']
        end=ent['end']
        label=ent['label']
        if label=="organization":
          corrected = self.correct_word(entity_text)
        else: corrected=entity_text
        corrected_entities.append({
          'original':entity_text,
          'corrected':corrected,
          'start':start,
          'end':end,
          'label':label
        })
      return corrected_entities

    def replace_entities_in_text(self,text,entities):
      entities=sorted(entities,key=lambda x:x['start'], reverse=True)
      for ent in entities:
        text= text[:ent['start']] + ent['corrected'] + text[ent['end']:]
      return text

file_path = 'companies.xlsx'
masker = DataMaskerCSV(file_path)
masker.csv_extraction()

# entities=masker.find_and_correct_entities("companies are accenure and wlrmart and they are doing fine")
# print(entities)
# corrected_text=masker.replace_entities_in_text("companies are accenure and wlrmart and they are doing fine",entities)
# print(corrected_text)


Fetching 4 files:   0%|          | 0/4 [00:00<?, ?it/s]

/usr/local/lib/python3.11/dist-packages/transformers/convert_slow_tokenizer.py:559: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


⏳ Faker data loaded in 1.802064 seconds
 mapping saved to: companies_mapping.json

⏳ Execution time anonymize_csv: 1.605085 seconds

⏳ Execution time csv_extraction: 4.240859 seconds


In [7]:
import json
import re
import time
from collections import defaultdict
import sqlparse
from sqlparse.sql import Token
from sqlparse.tokens import Literal,String
from gliner import GLiNER
from rapidfuzz import process, fuzz
class DbOperations:
    def __init__(self):
        self.map_path='companies_mapping.json'
        self.forward_mapping = defaultdict(dict)
        self.backward_mapping = defaultdict(dict)
        self.model=GLiNER.from_pretrained("urchade/gliner_base")
        with open(self.map_path, 'r') as f:
            data= json.load(f)
            self.forward_mapping = data.get('forward_mapping', {})
            self.backward_mapping = data.get('backward_mapping', {})
        self.entity_column_map={
        'name': 'company',
        'domain': 'url',
        }
    @staticmethod
    def time_it(func):
        def wrapper(*args, **kwargs):
            start = time.time()
            result = func(*args, **kwargs)
            end = time.time()
            print(f'\n⏳ Execution time {func.__name__}: {end-start:.6f} seconds')
            return result
        return wrapper
    @time_it
    def mask_sentence(self, sentence):
        entities=self.find_and_correct_entities(sentence)
        sentence= self.replace_entities_in_text(sentence,entities)
        print("corrected sentence: ",sentence)
        flat_map = {}
        for entity, value_map in self.forward_mapping.items():
            for original, fake in value_map.items():
                flat_map[original] = fake
        # Pre-lowercased lookup for fast replacement
        flat_map_lower = {k.lower(): v for k, v in flat_map.items()}
        # Identify which keys are present in the sentence (case-insensitive)
        sentence_lower = sentence.lower()
        matched_keys = [k for k in flat_map if k.lower() in sentence_lower]

        if matched_keys:
            # Sort matched keys by length (longest first) to avoid partial replacement
            matched_keys.sort(key=len, reverse=True)

            # Build regex pattern with alternation

            pattern = re.compile(
                r'(?<!\w)([\{\(\["\'\*\_]*?)(' +
                '|'.join(re.escape(k) for k in matched_keys) +
                r')([\}\)\]"\'\*\_]*?)(?!\w)',flags=re.IGNORECASE
            )
            def replace_match(match):
                prefix = match.group(1)  # e.g., '{' or '**'
                core = match.group(2)    # e.g., 'abc'
                suffix = match.group(3)  # e.g., '}' or '**'

                replaced = flat_map_lower.get(core.lower(), core)
                # print(f'{match.group(0)} => {prefix}{replaced}{suffix}')
                return f"{prefix}{replaced}{suffix}"
            sentence = pattern.sub(replace_match, sentence)

        return sentence

    @time_it
    def unmask_summary(self, sentence):
        flat_map = {}
        for entity, value_map in self.backward_mapping.items():
            for original, fake in value_map.items():
                flat_map[original] = fake
        # Pre-lowercased lookup for fast replacement
        flat_map_lower = {k.lower(): v for k, v in flat_map.items()}
        # Identify which keys are present in the sentence (case-insensitive)
        sentence_lower = sentence.lower()
        matched_keys = [k for k in flat_map if k.lower() in sentence_lower]

        if matched_keys:
            # Sort matched keys by length (longest first) to avoid partial replacement
            matched_keys.sort(key=len, reverse=True)

            # Build regex pattern with alternation
            pattern = re.compile(
                r'(?<!\w)([\{\(\["\'\*\_]*?)(' +
                '|'.join(re.escape(k) for k in matched_keys) +
                r')([\}\)\]"\'\*\_]*?)(?!\w)',flags=re.IGNORECASE
            )

            def replace_match(match):
                prefix = match.group(1)  # e.g., '{' or '**'
                core = match.group(2)    # e.g., 'abc'
                suffix = match.group(3)  # e.g., '}' or '**'

                replaced = flat_map_lower.get(core.lower(), core)
                # print(f'{match.group(0)} => {prefix}{replaced}{suffix}')
                return f"{prefix}{replaced}{suffix}"

            sentence = pattern.sub(replace_match, sentence)

        return sentence

    @time_it
    def query_mask(self, query):
        parsed = sqlparse.parse(query)
        masked_query = []

        for statement in parsed:
            tokens = list(statement.flatten())
            for token in tokens:
                original_value = token.value
                value = original_value.strip("\"'")  # remove both types of quotes
                replaced = False
                for ent in self.forward_mapping.values():
                    if value in ent:
                        fake_value = ent[value]
                        # Determine if the original was single or double quoted
                        if original_value.startswith("'") and original_value.endswith("'"):
                            token.value = f"'{fake_value}'"
                        elif original_value.startswith('"') and original_value.endswith('"'):
                            token.value = f'"{fake_value}"'
                        else:
                            token.value = fake_value

                        replaced = True
                        break

                masked_query.append(token.value)

        return ''.join(masked_query)
    @time_it
    def query_unmask(self, query):
        parsed = sqlparse.parse(query)
        masked_query = []

        for statement in parsed:
            tokens = list(statement.flatten())
            for token in tokens:
                original_value = token.value
                value = original_value.strip("\"'")  # remove both types of quotes
                replaced = False
                for ent in self.backward_mapping.values():
                    if value in ent:
                        fake_value = ent[value]
                        # Determine if the original was single or double quoted
                        if original_value.startswith("'") and original_value.endswith("'"):
                            token.value = f"'{fake_value}'"
                        elif original_value.startswith('"') and original_value.endswith('"'):
                            token.value = f'"{fake_value}"'
                        else:
                            token.value = fake_value

                        replaced = True
                        break

                masked_query.append(token.value)

        return ''.join(masked_query)

    @time_it
    def unmasking_results(self,results):
        de_anonymized = []
        for row in results:
            new_row={}
            for col,val in row.items():
                entity= self.entity_column_map.get(col.lower())
                key=f"{entity}"
                if key in self.backward_mapping and val in self.backward_mapping[key]:
                    new_row[col]=self.backward_mapping[key][val]
                else:
                    new_row[col]=val
            de_anonymized.append(new_row)
        return de_anonymized

    @time_it
    def masking_results(self,results):
        de_anonymized = []
        for row in results:
            new_row={}
            for col,val in row.items():
                entity= self.entity_column_map.get(col.lower())
                key=f"{entity}"
                if key in self.forward_mapping and val in self.forward_mapping[key]:
                    new_row[col]=self.forward_mapping[key][val]
                else:
                    new_row[col]=val
            de_anonymized.append(new_row)
        return de_anonymized
    @time_it
    def correct_word(self,word,threshold=65):
      valid_list=[company.lower() for company in self.forward_mapping['company'].keys()]
      match=process.extractOne(word,valid_list,scorer=fuzz.ratio)
      return match[0] if match and match[1]>=threshold else word
    @time_it
    def find_and_correct_entities(self,text):
      valid_company=[company.lower() for company in self.forward_mapping['company'].keys()]
      entities=self.model.predict_entities(text,labels=["person","organization,'location"])

      corrected_entities=[]
      for ent in entities:
        entity_text=ent['text']
        start=ent['start']
        end=ent['end']
        label=ent['label']
        if label=="organization":
          corrected = self.correct_word(entity_text)
        elif label=="location":
          corrected = self.correct_word(entity_text)
        elif label=="person":
          corrected = self.correct_word(entity_text)
        else: corrected=entity_text
        corrected_entities.append({
          'original':entity_text,
          'corrected':corrected,
          'start':start,
          'end':end,
          'label':label
        })
      return corrected_entities

    @time_it
    def replace_entities_in_text(self,text,entities):
      entities=sorted(entities,key=lambda x:x['start'], reverse=True)
      for ent in entities:
        text= text[:ent['start']] + ent['corrected'] + text[ent['end']:]
      return text

# Example usage
op=DbOperations()
text="companies : tata consultensy sercices and wlrmart and ibm"
masked=op.mask_sentence(text)
print(masked)


Fetching 4 files:   0%|          | 0/4 [00:00<?, ?it/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.



⏳ Execution time find_and_correct_entities: 0.697122 seconds

⏳ Execution time replace_entities_in_text: 0.000016 seconds
corrected sentence:  companies : tata consultensy sercices and wlrmart and ibm

⏳ Execution time mask_sentence: 0.873330 seconds
companies : Nguyen-Alvarado Ltd consultensy sercices and wlrmart and Williams-Waller Co


In [2]:
from collections import Counter
from gliner import GLiNER
model = GLiNER.from_pretrained("urchade/gliner_multi_pii-v1")
import pandas as pd
import time




/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Fetching 4 files:   0%|          | 0/4 [00:00<?, ?it/s]

README.md:   0%|          | 0.00/3.04k [00:00<?, ?B/s]

gliner_config.json:   0%|          | 0.00/478 [00:00<?, ?B/s]

.gitattributes:   0%|          | 0.00/1.52k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.16G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/579 [00:00<?, ?B/s]

spm.model:   0%|          | 0.00/4.31M [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/transformers/convert_slow_tokenizer.py:559: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


In [10]:
import re
@staticmethod
def time_it(func):
    def wrapper(*args, **kwargs):
        start = time.time()
        result = func(*args, **kwargs)
        end = time.time()
        print(f'\n⏳ Execution time {func.__name__}: {end-start:.6f} seconds')
        return result
    return wrapper
def get_most_common_entity(results):
    label_counter = Counter()
    for entity in results:
        label = entity.get("label")
        if label:
            label_counter[label] += 1

    if label_counter:
        print(label_counter)
        # Get label with highest count
        most_common_label, _ = label_counter.most_common(1)[0]
        # print(most_common_label)
        if label_counter[most_common_label]>=5:
            return most_common_label
    return None
@time_it
def analyze_column(df):
    entity_columns = {}
    keywords = ["description", "remarks", "notes", "comments", "observations", "details", "summary", "explanation",
    "reviews", "feedback", "testimonials", "opinions", "assessment", "suggestions", "experience","status",
    "incident_report", "case_notes", "audit_notes", "findings", "status_update", "history", "progress_report",
    "additional_info", "clarifications", "justification", "annotations", "excerpts", "statement", "explanation_text","reason"]
    des=[col for col in df.columns if any(re.search(keyword, col, re.IGNORECASE) for keyword in keywords)]
    print("descriptive: ",des)
    for col in df.columns:
      if "id" in col.lower():
        entity_columns[col] = "ID"
      elif "date" in col.lower():
        entity_columns[col] = "date"
      elif col in des:
        entity_columns[col] = "description"
      else:
        print("colum: ",col)
        values = df[col].dropna().astype(str).tolist()[:10]
        if not values:
            continue
        combined_text = " , ".join(values)  # You could also use newline or space
        results_batch = model.predict_entities(combined_text, labels=["person","city", "phone number", "location", "email", "url", "company","country"])
        print(results_batch)
        entity_count = {}
        if not results_batch:
            continue
        else:
          most_common_entity = get_most_common_entity(results_batch)
          if most_common_entity:
              entity_columns[col] = most_common_entity

    return entity_columns


In [11]:
# df = pd.read_csv('companies_100k.csv')
df1=pd.read_excel("SO(real).xlsx",nrows=200)
analyze_column(df1)

descriptive:  ['RR Status', 'UST Role Description', 'Job Description', 'Notes for WFM or TA', 'Cancelled Reasons', 'Resubmitted Reason2', 'MTE Status', 'External Status', 'CRM Status', 'Status', 'Comments', 'Reasons']
colum:  RR Type
[]
colum:  Priority
[{'start': 0, 'end': 2, 'text': 'P4', 'label': 'location', 'score': 0.5399155616760254}, {'start': 5, 'end': 7, 'text': 'P4', 'label': 'location', 'score': 0.5458663105964661}, {'start': 10, 'end': 12, 'text': 'P4', 'label': 'location', 'score': 0.5088265538215637}, {'start': 20, 'end': 22, 'text': 'P4', 'label': 'location', 'score': 0.547615647315979}, {'start': 25, 'end': 27, 'text': 'P4', 'label': 'location', 'score': 0.6016972661018372}, {'start': 30, 'end': 32, 'text': 'P4', 'label': 'location', 'score': 0.5364493727684021}, {'start': 40, 'end': 42, 'text': 'P4', 'label': 'location', 'score': 0.5534209609031677}, {'start': 45, 'end': 47, 'text': 'P4', 'label': 'location', 'score': 0.6973801851272583}]
Counter({'location': 8})
colum

{'Resource Request ID': 'ID',
 'RR Status': 'description',
 'Priority': 'location',
 'City': 'city',
 'State': 'location',
 'Country': 'country',
 'Altenate Location': 'location',
 'RR Start Date': 'date',
 'RR End Date': 'date',
 'Account Name': 'company',
 'Project ID': 'ID',
 'WFME': 'person',
 'WFME ID': 'ID',
 'HM': 'person',
 'HM ID': 'ID',
 'AM': 'person',
 'AM ID': 'ID',
 'Replacement Type': 'person',
 'UST Role Description': 'description',
 'Job Description': 'description',
 'Notes for WFM or TA': 'description',
 'Project Start Date': 'date',
 'Project End Date': 'date',
 'RR Finance Approved Date': 'date',
 'WFM Approved Date': 'date',
 'Cancelled Reasons': 'description',
 'Edit Requested Date': 'date',
 'Resubmitted Date': 'date',
 'Resubmitted Reason2': 'description',
 'Recruiter Name': 'person',
 'Recruiter ID': 'ID',
 'Last Updated On': 'date',
 'Last Activity Date': 'date',
 'Hiring request Submit Date (MTE)': 'date',
 'MTE Status': 'description',
 'SO Initiator Name': '

In [60]:
# from collections import Counter

# def get_most_common_entity(results):
#     label_counter = Counter()
#     for entity in results:
#         label = entity.get("label")
#         if label:
#             label_counter[label] += 1

#     if label_counter:
#         # Get label with highest count
#         most_common_label, _ = label_counter.most_common(1)[0]
#         return most_common_label
#     return None
# def analyze_column(df):
#     entity_columns = {}

#     for col in df.columns:
#         values = df[col].dropna().astype(str).unique().tolist()[:10]
#         if not values:
#             continue
#         combined_text = " | ".join(values)  # You could also use newline or space
#         results_batch = model.predict_entities(f'"{combined_text}"', labels=["person", "phone", "location", "email", "url", "company"])
#         print(results_batch)
#         entity_count = {}
#         if not results_batch:
#             continue
#         else:
#           most_common_entity = get_most_common_entity(results_batch)
#           if most_common_entity:
#               entity_columns[col] = most_common_entity

#     return entity_columns

In [68]:
result=model.predict_entities("['ibm.com','ibm','tcs.com','I work at UST global','Midhun']",labels=["person","phone","location","email","url",'company'])
result

[{'start': 2,
  'end': 9,
  'text': 'ibm.com',
  'label': 'url',
  'score': 0.9080606698989868},
 {'start': 12,
  'end': 15,
  'text': 'ibm',
  'label': 'company',
  'score': 0.9255616664886475},
 {'start': 18,
  'end': 25,
  'text': 'tcs.com',
  'label': 'url',
  'score': 0.9852312803268433},
 {'start': 38,
  'end': 48,
  'text': 'UST global',
  'label': 'company',
  'score': 0.6665892601013184},
 {'start': 51,
  'end': 57,
  'text': 'Midhun',
  'label': 'person',
  'score': 0.9854029417037964}]

To learn more about accelerating pandas on Colab, see the [10 minute guide](https://colab.research.google.com/github/rapidsai-community/showcase/blob/main/getting_started_tutorials/cudf_pandas_colab_demo.ipynb) or
 [US stock market data analysis demo](https://colab.research.google.com/github/rapidsai-community/showcase/blob/main/getting_started_tutorials/cudf_pandas_stocks_demo.ipynb).